# 논문 토큰 호출용 클래스 정의

In [2]:
import json, base64, requests
from Crypto.Cipher import AES

class AESTestClass:
    def __init__(self, plain_txt, key):
        # iv, block_size 값은 고정
        self.iv = 'jvHJ1EFA0IXBrxxz'
        self.block_size = 16
        self.plain_txt = plain_txt
        self.key = key

    def pad(self):
        # PKCS#7 패딩
        number_of_bytes_to_pad = self.block_size - len(self.plain_txt) % self.block_size
        ascii_str = chr(number_of_bytes_to_pad)
        padding_str = number_of_bytes_to_pad * ascii_str
        return self.plain_txt + padding_str

    def encrypt(self):
        cipher = AES.new(self.key.encode('utf-8'), AES.MODE_CBC, self.iv.encode('utf-8'))
        padded_txt = self.pad()
        encrypted_bytes = cipher.encrypt(padded_txt.encode('utf-8'))
        # URL-safe Base64
        encrypted_str = base64.urlsafe_b64encode(encrypted_bytes).decode('utf-8')
        return encrypted_str

# 토큰 호출 함수

In [3]:
from datetime import datetime
# 나중에 AESTestClass도 호출해야 함.

def call_access_token(MAC_ADDRESS, API_KEY, CLIENT_ID):
    # 맥주소
    mac = (MAC_ADDRESS or "").strip().strip('"').strip("'").upper().replace(":", "-")
    if not mac:
        raise SystemExit("MAC_ADDRESS가 비어있음")

    # datetime 생성
    dt = datetime.now().strftime('%Y%m%d%H%M%S')

    # JSON 페이로드 생성
    payload = {
        "mac_address": mac,
        "datetime": dt
    }
    plain_json = json.dumps(payload, separators=(',', ':'))

    # AES 암호화 → Base64
    aes = AESTestClass(plain_txt=plain_json, key=API_KEY)
    b64_cipher = aes.encrypt()

    # 인코딩 + 토큰 요청
    endpoint = "https://apigateway.kisti.re.kr/tokenrequest.do"
    params = {
        "accounts": b64_cipher,
        "client_id": CLIENT_ID
    }

    response = requests.get(endpoint, params=params, timeout=10)
    response.raise_for_status()

    data = response.json()
    token = data['access_token'] 
    return token

# 논문 API 호출 함수

In [4]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
from dotenv import load_dotenv
load_dotenv(override=True)

# 관련 논문 검색 함수
def ARTI_search(client_id, token, query):
    url = "https://apigateway.kisti.re.kr/openapicall.do"
    params = {
        "client_id": client_id,
        "token": token,
        "version": 1.0,
        "action": "search",
        "target": "ARTI",
        "searchQuery": query,
        'curPage': 1, # 현재페이지 번호
        'rowCount': 10, # 디스플레이 건수(기본값 10, 최대값 100)

    }
    print("🔎 호출 URL 미리보기:", requests.Request('GET', url, params=params).prepare().url)
    res = requests.get(url, params=params, timeout=20)
    xml = res.text
    
    return xml


# 단일 논문 조회 함수
def ARTI_browse(client_id, token, cn):
    
    url = "https://apigateway.kisti.re.kr/openapicall.do"
    params = {
        "client_id": client_id,
        "token": token,
        "version": 1.0,
        "action": "browse",
        "target": "ARTI",
        "cn": cn,
        "include": "",
        "exclude": None,
    }
    
    print("🔎 호출 URL 미리보기:", requests.Request('GET', url, params=params).prepare().url)
    res = requests.get(url, params=params, timeout=20)
    xml = res.text
    
    return xml

def xml_to_df(xml):
    # XML 파싱
    root = ET.fromstring(xml)

    # recordList 찾기
    record_list_element = root.find('recordList')

    # 데이터를 담을 리스트
    records = []

    if record_list_element is not None:
        # 각 record에 대해 반복
        for record_element in record_list_element.findall('record'):
            record_data = {}
            # 각 item에 대해 반복
            for item_element in record_element.findall('item'):
                meta_code = item_element.get('metaCode')
                # CDATA 섹션의 텍스트 추출
                value = item_element.text.strip() if item_element.text else ''
                record_data[meta_code] = value
            records.append(record_data)

    df = pd.DataFrame(records)
    return df

# 논문 가져오기

In [5]:
import os, xmltodict
from dotenv import load_dotenv

load_dotenv(override=True)

CLIENT_ID = os.getenv("SCIENCEON_CLIENT_ID")
ARTI_KEY = os.getenv("SCIENCEON_API_KEY")
MAC_ADDRESS = os.getenv("MAC_ADDRESS")

access_token = call_access_token(MAC_ADDRESS, ARTI_KEY, CLIENT_ID)

cn = 'JAKO200411922932805'

xml = ARTI_browse(CLIENT_ID, access_token, cn)

dict_data = xmltodict.parse(xml)
with open("../data/input_data.json", "w", encoding="utf-8") as f:
    json.dump(dict_data, f, ensure_ascii=False, indent=4)

df = xml_to_df(xml)

print(df['Title'].iloc[0])
print(df['Abstract'].iloc[0])

🔎 호출 URL 미리보기: https://apigateway.kisti.re.kr/openapicall.do?client_id=5e9c4c6079d323f1a6accb980253cac8733c0249be263f11da5fbc9b2f7cb58d&token=639ef938d40657d74c173422644d9945a6f2655f05d18ec61b32916a0c4327e0&version=1.0&action=browse&target=ARTI&cn=JAKO200411922932805&include=
한국인의 3차원 무릎관절 구축 및 형상 측정
It is necessary to have a model that describes the feature of the knee Joint with a sufficient accuracy. Koreans, however, do not have their own knee joint model to be used in the total knee replacement arthroplasty. They have to use European or American models which do not match Koreans. Three-dimensional visualization techniques are found to be useful in a wide range of medical applications. Three-dimensional imaging studies such as CT(computed tomography) and MRI(magnetic resonance image) provide the primary source of patient-specific data. Three-dimensional knee joint models were constructed by image processing of the CT data of 10 subjects. Using the constructed model, the dimensions 

In [6]:
def transform_query(input_query):

    query = {
        "BI": input_query,  # 전체
        # "TI": None,  # 논문명
        # "AU": None,  # 저자
        # "AB": None,  # 초록
        # "KW": None,  # 키워드
        # "PB": None,  # 출판사(발행기관)
        # "SN": None,  # ISSN
        # "BN": None,  # ISBN
        # "PY": None,  # 발행년도
        # "CN": None,  # 문헌번호
        # "DI": None   # DOI
    }

    json_query = json.dumps(query, separators=(',', ':')) 

    return json_query

In [7]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

CLIENT_ID = os.getenv("SCIENCEON_CLIENT_ID")
ARTI_KEY = os.getenv("SCIENCEON_API_KEY")
MAC_ADDRESS = os.getenv("MAC_ADDRESS")

access_token = call_access_token(MAC_ADDRESS, ARTI_KEY, CLIENT_ID)

queries =  ['3D knee model', 'Korean knee joint', 'knee joint measurement', 'CT knee imaging', 'knee arthroplasty model']


df = pd.DataFrame()

for query in queries:
    xml = ARTI_search(CLIENT_ID, access_token, transform_query(query))
    tmp = xml_to_df(xml)
    tmp["query"] = query
    df = pd.concat([df, tmp], ignore_index=True)

df.to_csv('../data/search_results_article.csv', index=False, encoding='utf-8')

display(df)

🔎 호출 URL 미리보기: https://apigateway.kisti.re.kr/openapicall.do?client_id=5e9c4c6079d323f1a6accb980253cac8733c0249be263f11da5fbc9b2f7cb58d&token=b741b4b82334b5de6b018b5cfd368fcdadc35c3895aa6fcf5fc204b41f97c977&version=1.0&action=search&target=ARTI&searchQuery=%7B%22BI%22%3A%223D+knee+model%22%7D&curPage=1&rowCount=10
🔎 호출 URL 미리보기: https://apigateway.kisti.re.kr/openapicall.do?client_id=5e9c4c6079d323f1a6accb980253cac8733c0249be263f11da5fbc9b2f7cb58d&token=b741b4b82334b5de6b018b5cfd368fcdadc35c3895aa6fcf5fc204b41f97c977&version=1.0&action=search&target=ARTI&searchQuery=%7B%22BI%22%3A%22Korean+knee+joint%22%7D&curPage=1&rowCount=10
🔎 호출 URL 미리보기: https://apigateway.kisti.re.kr/openapicall.do?client_id=5e9c4c6079d323f1a6accb980253cac8733c0249be263f11da5fbc9b2f7cb58d&token=b741b4b82334b5de6b018b5cfd368fcdadc35c3895aa6fcf5fc204b41f97c977&version=1.0&action=search&target=ARTI&searchQuery=%7B%22BI%22%3A%22knee+joint+measurement%22%7D&curPage=1&rowCount=10
🔎 호출 URL 미리보기: https://apigateway.kisti

,CN,DBCode,JournalId,Publisher,JournalName,ISSN,ISBN,VolumeId,VolNo1,VolNo2,...,FulltextFlag,AbstractFlag,PageInfo,DOI,FulltextURL,ContentURL,MobileURL,Keyword,Degree,query
0,NART37692760,JAFO,NJOU00035957,,日本機械學會論文集. Transactions of the japan society o...,0387-5024;1884-8354;,,374,72,713,...,2,1,pp.153-160,,,http://click.ndsl.kr/servlet/OpenAPIDetailView...,http://click.ndsl.kr/servlet/OpenAPIDetailView...,Three Dimensional Model . Tibio-Femoral Joint ...,,3D knee model
1,NPAP13614853,CFFO,NPRO00387217,IEEE,"Computer Systems and Applications (AICCSA), 20...",;2161-5330;,9781665409698;;,1,2021,11,...,2,1,pp.1-7,,,http://click.ndsl.kr/servlet/OpenAPIDetailView...,http://click.ndsl.kr/servlet/OpenAPIDetailView...,,,3D knee model
2,NART74557425,JAFO,NJOU00040020,,Computer methods in biomechanics and biomedica...,1025-5842;1476-8259;,,193,18,7,...,2,1,pp.721-729,http://dx.doi.org/10.1080/10255842.2013.843676,,http://click.ndsl.kr/servlet/OpenAPIDetailView...,http://click.ndsl.kr/servlet/OpenAPIDetailView...,3D knee model . fluoroscopic images . knee . s...,,3D knee model
3,NPAP13526087,CFFO,NPRO00386177,IEEE,"Robotics and Automation (ICRA), 2021 IEEE Inte...",;2577-087x;,9781728190778;;,1,2021,5,...,2,1,pp.12501-12507,,,http://click.ndsl.kr/servlet/OpenAPIDetailView...,http://click.ndsl.kr/servlet/OpenAPIDetailView...,,,3D knee model
4,NART37654746,JAFO,NJOU00298280,,IFMBE proceedings,1680-0737;;,,7,14,5,...,0,0,pp.3177-3181,,,http://click.ndsl.kr/servlet/OpenAPIDetailView...,http://click.ndsl.kr/servlet/OpenAPIDetailView...,,,3D knee model
5,NART92905321,JAFO,NJOU00298818,"Trans Tech Publications, Ltd.",Applied mechanics and materials,1660-9336;1662-7482;,,867,333,,...,2,1,pp.934-937,http://dx.doi.org/10.4028/www.scientific.net/A...,,http://click.ndsl.kr/servlet/OpenAPIDetailView...,http://click.ndsl.kr/servlet/OpenAPIDetailView...,,,3D knee model
6,NART07397859,JAFO,NJOU00024228,,BED,1521-4613;;,,10,39,,...,0,0,pp.1-2,,,http://click.ndsl.kr/servlet/OpenAPIDetailView...,http://click.ndsl.kr/servlet/OpenAPIDetailView...,,,3D knee model
7,NART74050161,JAFO,NJOU00299641,,法醫學雜誌 = Journal of forensic medicine,1004-5619;;,,74,30,1,...,2,1,pp.1-6,,,http://click.ndsl.kr/servlet/OpenAPIDetailView...,http://click.ndsl.kr/servlet/OpenAPIDetailView...,,,3D knee model
8,NART135717037,JAFO,NJOU00406841,Public Library of Science,PLoS ONE,;1932-6203;,,233,20,6,...,2,1,pp.e0324912,https://doi.org/10.1371/journal.pone.0324912,,http://click.ndsl.kr/servlet/OpenAPIDetailView...,http://click.ndsl.kr/servlet/OpenAPIDetailView...,,,3D knee model
9,NART113545590,JAFO,NJOU00035957,The Japan Society of Mechanical Engineers,日本機械學會論文集. Transactions of the japan society o...,0387-5024;1884-8354;,,374,72,713,...,2,0,pp.153-160,http://dx.doi.org/10.1299/kikaic.72.153,,http://click.ndsl.kr/servlet/OpenAPIDetailView...,http://click.ndsl.kr/servlet/OpenAPIDetailView...,,,3D knee model


In [8]:
cleaned_df = (
    df[
        ['CN', 'Title', 'Abstract', 'Pubyear', 'Author', 'ContentURL', 'query']
    ]
    .rename(
        columns={
            'CN': 'ID',
            'Title': 'title',
            'Abstract': 'description',
            'Pubyear': 'pubyear',
            'Author': 'author',
            'ContentURL': 'URL'
        }
    )
)

cleaned_df['category'] = 'article'


display(cleaned_df)

,ID,title,description,pubyear,author,URL,query,category
0,NART37692760,A 3D Model Analysis of Knee Prosthetic Motion ...,<P>The object of this study is to introduce co...,2006,"KODA, Takefumi;KATSUHARA, Tadasuke;HIROKAWA, S...",http://click.ndsl.kr/servlet/OpenAPIDetailView...,3D knee model,article
1,NPAP13614853,A Novel Deep Learning Model for Knee Cartilage...,"<P>Over the past few years, osteoarthritis is ...",2021,"Hedi Bedoui, Mohamed",http://click.ndsl.kr/servlet/OpenAPIDetailView...,3D knee model,article
2,NART74557425,Principal component analysis in construction o...,<P>The statistical shape model (SSM) method th...,2015,"Tsai, Tsung-Yuan;Li, Jing-Sheng;Wang, Shaobai;...",http://click.ndsl.kr/servlet/OpenAPIDetailView...,3D knee model,article
3,NPAP13526087,A 3D Printed Mechanical Model of the Knee to D...,"<P>In this article, a novel 3D printed knee mo...",2021,"Rottinger, Timon;Rottinger, Heinz;",http://click.ndsl.kr/servlet/OpenAPIDetailView...,3D knee model,article
4,NART37654746,A 3D model analysis for developing multifuncti...,,2007,"Katsuhara, T.;Sakaguchi, J.;Hirokawa, S.;",http://click.ndsl.kr/servlet/OpenAPIDetailView...,3D knee model,article
5,NART92905321,Study on 3D Model Reconstruction of Human Knee...,<P>Reconstruction of a 3D model for human knee...,2013,"Han, Yue Mei",http://click.ndsl.kr/servlet/OpenAPIDetailView...,3D knee model,article
6,NART07397859,Development of a 3D Computational Human Knee J...,,1998,"Gil, J.;Li, G.;Kanamori, A.;Woo, S. L.-Y.;",http://click.ndsl.kr/servlet/OpenAPIDetailView...,3D knee model,article
7,NART74050161,3D finite element model of human knee injuries...,<P>To explore the injury mechanism of the huma...,2014,"Huang, Wei-Hua;Huang, Ping;Li, Zheng-Dong;Zou,...",http://click.ndsl.kr/servlet/OpenAPIDetailView...,3D knee model,article
8,NART135717037,Same-model and cross-model variability in knee...,<P>Purpose<P>Magnetic Resonance Imaging (MRI) ...,2025,"Katano, Hisako;Kaneko, Haruka;Sasaki, Eiji;Has...",http://click.ndsl.kr/servlet/OpenAPIDetailView...,3D knee model,article
9,NART113545590,A 3D Model Analysis of Knee Prosthetic Motion ...,,2006,"KODA, Takefumi;KATSUHARA, Tadasuke;HIROKAWA, S...",http://click.ndsl.kr/servlet/OpenAPIDetailView...,3D knee model,article
